In [1]:
import os
import pickle
import jsonlines
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm
from zhipuai import ZhipuAI

In [2]:
data = json.load(open("./handled/id_map.json", "r"))

In [3]:
def construct_prompt(label_list, domain="A"):

    if domain == "A":
        prompt = "书籍有如下的标签\n:"
    elif domain == "B":
        prompt = "电影有如下的标签\n:"
    elif domain == "C":
        prompt = "音乐有如下的标签\n:"

    for label in label_list:
        prompt += "{}, ".format(label)
    
    if len(prompt) > 100:
        prompt = prompt[:100]
    else:
        prompt = prompt[:-2]

    return prompt

In [ ]:
item_text_A = []
label_num = []

for labels in tqdm(data["item_dict"]["0"]["id2label"].values()):
    label_list = []
    for label in labels:
        label_list += label.split("|")
    label_list = list(set(label_list))  # remove replicated label
    label_num.append(len(label_list))
    item_text_A.append(construct_prompt(label_list, "A"))

In [ ]:
item_text_B = []
label_num = []

for labels in tqdm(data["item_dict"]["1"]["id2label"].values()):
    label_list = []
    for label in labels:
        label_list += label.split("|")
    label_list = list(set(label_list))  # remove replicated label
    label_num.append(len(label_list))
    item_text_B.append(construct_prompt(label_list, "B"))

In [6]:
def construct_title(label_list):

    count = 0
    label_str = ""
    for label in label_list:
        label_str += label
        label_str += ", "
        count += 1

        if count >= 5:
            break
    
    title = "<{}>".format(label_str)

    return title

In [ ]:
title_list = []
for labels in tqdm(data["item_dict"]["0"]["id2label"].values()):
    # title = construct_title(labels)
    # title_list.append(title)
    title_list.append(labels)

with open("./handled/title_A.pkl", "wb") as f:
    pickle.dump(title_list, f)

In [ ]:
len(title_list)

In [ ]:
title_list = []
for labels in tqdm(data["item_dict"]["1"]["id2label"].values()):
    # title = construct_title(labels)
    # title_list.append(title)
    title_list.append(labels)

with open("./handled/title_B.pkl", "wb") as f:
    pickle.dump(title_list, f)

In [ ]:
len(title_list)

In [ ]:
data["item_dict"]["0"]["id2str"].keys()

In [ ]:
# 统计domainA和B的平均prompt长度，以及空prompt的占比
lenA, lenB = [], []
voidA, voidB = 0, 0

for item_text in item_text_A:
    if len(item_text) < 11:
        voidA += 1
    else:
        lenA.append(len(item_text))

for item_text in item_text_B:
    if len(item_text) < 12:
        voidB += 1
    else:
        lenB.append(len(item_text))

print("The ratio of void item text in domain A: %.5f, domain B: %.5f" % 
      (voidA / len(item_text_A), voidB / len(item_text_B)))
print("The average length of domain A: %.5f, domain B: %.5f" %
      (np.mean(lenA), np.mean(lenB)))

In [9]:
def save_data(data_path, data):
    '''write all_data list to a new jsonl'''
    with jsonlines.open("./handled/"+ data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

# for book domain
id_map = data["item_dict"]["0"]["id2str"]
json_data = []
for id, item_prompt in enumerate(item_text_A):
    item_id = str(id+1)
    json_data.append({"input": item_prompt, "target": "", "item": id_map[item_id], "item_id": item_id})

# save_data("item_str_A.jsonline", json_data)

# for movie domain
id_map = data["item_dict"]["1"]["id2str"]
json_data = []
for id, item_prompt in enumerate(item_text_B):
    item_id = str(id+1)
    json_data.append({"input": item_prompt, "target": "", "item": id_map[item_id], "item_id": item_id})

save_data("/item_str_B.jsonline", json_data)

In [25]:
with open("./handled/item_prompt.pkl", "wb") as f:
    pickle.dump((item_text_A, item_text_B), f)

In [ ]:
client = ZhipuAI(api_key="") 

In [ ]:
item_emb_A = []

for item_text in tqdm(item_text_A):
    response = client.embeddings.create(
        model="embedding-3", #填写需要调用的模型编码
        input=item_text
    )

    item_emb_A.append(response.data[0].embedding)

In [46]:
item_emb_A = np.array(item_emb_A)

with open("./handled/item_emb_A.pkl", "wb") as f:
    pickle.dump(item_emb_A, f)

In [ ]:
item_emb_B = []

for item_text in tqdm(item_text_B):
    response = client.embeddings.create(
        model="embedding-3", #填写需要调用的模型编码
        input=item_text
    )

    item_emb_B.append(response.data[0].embedding)

In [48]:
item_emb_B = np.array(item_emb_B)

with open("./handled/item_emb_B.pkl", "wb") as f:
    pickle.dump(item_emb_B, f)